<a href="https://colab.research.google.com/github/WongKingLongKen/LearnSomeBlockChain/blob/main/blockchain_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference: https://drlee.io/building-your-own-blockchain-in-python-a-step-by-step-guide-ec10ea6c976d

# Step 1: create hash

In [2]:
import hashlib
import time

class Block:
  def __init__(self, timestamp: str, transactions, prior_hash=''):
    # self.index = index
    self.transactions = transactions
    self.timestamp = timestamp
    self.prior_hash = prior_hash
    self.nonce = 0
    self.hash = self.create_hash()

  def create_hash(self):
    block_string = (str(self.prior_hash) + str(self.timestamp) +
      str(self.transactions) + str(self.nonce)).encode()
    # Return hash using SHA-256 (hashing algo) so that the fixed-length has string is unique to the data within the block
    return hashlib.sha256(block_string).hexdigest()

  def mine_block(self, difficulty):
    target = '0' * difficulty
    while self.hash[:difficulty] != target:
        self.nonce += 1
        self.hash = self.create_hash()
    print(f"Block mined! Nonce: {self.nonce}, Hash: {self.hash}")



Encryption is two-way algorithm while Hashing is one-way. Encryption includes encrypt and decrypt while Hashing is irreversible. Hashing is better for storing a password. PW -> Hash but never Hash -> PW.

A salt is a random piece of data added to the file to ensure uniqueness, e.g. Hash(data+salt): Hash(qwerty + m879) = Hash1.

PW -> Hash -> Encrypt.

# Step 2: assemble the blocks into a chain

The first block in any blockchain is called genesis block. It's unique and doesn't have any prior block to refer to, so its prior_hash is set to a default value.

In [3]:
class ManageBlkChain:
  def __init__(self):
    self.chain = [self.create_genesis_block()]
    self.difficulty = 4 # mining difficulty
    self.pending_transactions = []
    self.mining_reward = 10


  def create_genesis_block(self):
    # return Block(0, '11/20/1978', 'BlockchainTrainingAlliance.com', '0')
    return Block(time.time(), [], "0")

  def get_last_block(self):
    return self.chain[-1]

  def mine_pending_transactions(self, mining_reward_address):
    block = Block(time.time(), self.pending_transactions, self.get_last_block().hash)
    # newly mined block
    self.chain.append(block)
    self.pending_transactions = [Transaction(None, mining_reward_address, self.mining_reward)]

  def create_transaction(self, transaction):
    self.pending_transactions.append(transaction)

  def get_balance_of_address(self, address):
    balance = 0

    for block in self.chain:
      for transaction in block.transactions:
        if transaction.from_addr == address:
          balance -= transaction.amount
        if transaction.to_addr == address:
          balance += transaction.amount

    return balance

  def add_block(self, new_block):
    new_block.prior_hash = self.get_last_block().hash
    new_block.mine_block(self.difficulty)
    # new_block.hash = new_block.create_hash()
    self.chain.append(new_block)

  def is_bc_valid(self):
    for idx in range(1, len(self.chain)):
      current_block = self.chain[idx]
      previous_block = self.chain[idx - 1]

      if current_block.hash != current_block.create_hash():
        return False

      if current_block.prior_hash != previous_block.hash:
        return False

    return True

In [ ]:
class Transaction:
  def __init__(self, from_addr, to_addr, amount):
    self.from_addr = from_addr
    self.to_addr = to_addr
    self.amount = amount

Example

In [6]:
coin = ManageBlkChain()

# coin.add_block(Block(1, '11/21/1978', 'amount = 10'))
# coin.add_block(Block(2, '11/22/1978', 'amount = 40'))
# coin.add_block(Block(3, '11/23/1978', 'amount = 20'))
# coin.add_block(Block(4, '11/24/1978', 'amount = 30'))

coin.create_transaction(Transaction('address1', 'address2', 70))
coin.create_transaction(Transaction('address2', 'address1', 40))

print("Starting mining process...")
coin.mine_pending_transactions('miner-address')

print(f"\nBalance of miner's wallet: {coin.get_balance_of_address('miner-address')}")

print("Mining again to receive the reward...")
coin.mine_pending_transactions('miner-address')
print(f"\nBalance of miner's wallet after second mining: {coin.get_balance_of_address('miner-address')}")
# Print the blockchain to the console
# import json
# print(json.dumps(coin.chain, default=lambda o: o.__dict__, indent=4))

Starting mining process...

Balance of miner's wallet: 0
Mining again to receive the reward...

Balance of miner's wallet after second mining: 10


# Step 3: validate the blockchain

Look back to class ManageBlkChain

In [7]:
# Test for checking the built-up of validation part by tampering
print('Is ManageBlkChain Valid? ' + str(coin.is_bc_valid()))
# After tampering
coin.chain[1].data = 'amount = 1,000'
print('Is ManageBlkChain Valid after tampering? ' + str(coin.is_bc_valid()))
# Update the hash to match the tampered data
coin.chain[1].hash = coin.chain[1].create_hash()
print('Is ManageBlkChain Valid after updating the hash? ' + str(coin.is_bc_valid()))

Is ManageBlkChain Valid? True
Is ManageBlkChain Valid after tampering? True
Is ManageBlkChain Valid after updating the hash? True


# Step 4: Proof of Work, to prevent blocks from being added too quickly that may result in spam or unauthorized changes

Check the first step and second step

# Step 5: Transaction Class

Check the first step and be aware of the import of time, also Transaction class